In [ ]:
!pip install faker pandarallel psycopg2

import hashlib
import os
import shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from faker import Faker
from IPython.core.display import display, HTML
import pandas as pd
from pandarallel import pandarallel
from pandarallel.utils.progress_bars import ProgressBarsNotebookLab
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm.session import sessionmaker
from tempfile import NamedTemporaryFile
from tqdm.auto import tqdm

In [ ]:
Base = declarative_base()
pandarallel.initialize(progress_bar=True, verbose=0)
tqdm.pandas()

db_name = os.getenv('DB_NAME')
db_user = os.getenv('POSTGRES_USER')
db_password = os.getenv('POSTGRES_PASSWORD')
db_host = 'db'

%env PGPASSWORD=$db_password

Session = sessionmaker()
db = create_engine(f"postgres://{db_user}:{db_password}@{db_host}/{db_name}")
Session.configure(bind=db)

def fake_email(fake, first_name, last_name):
    pattern_choice = fake.pyint(min_value=0, max_value=3)
    if pattern_choice == 0:
        return f"{first_name.lower()}.{last_name.lower()}@{fake.free_email_domain()}"
    elif pattern_choice == 1:
        return f"{first_name[0].lower()}{last_name.lower()}@{fake.free_email_domain()}"
    elif pattern_choice == 2:
        return f"{first_name.lower()}@{last_name.lower()}.{fake.tld()}"
    elif pattern_choice == 3:
        return f"{last_name.lower()}{first_name[0].lower()}@{fake.domain_name()}"

class Invitation(Base):
    __tablename__ = 'invitations'
    id = Column(Integer, primary_key=True)
    email = Column(String)
    first_name = Column(String)
    last_name = Column(String)
    
    def anonymize_row(row):
        fake = Faker()
        Faker.seed(row['id'])
        first_name = fake.first_name()
        last_name = fake.last_name()
        row['first_name'] = first_name
        row['last_name'] = last_name
        row['email'] = fake_email(fake, first_name, last_name)
        return row

class Project(Base):
    __tablename__ = 'projects'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    phone = Column(String)
    
    def anonymize_row(row):
        fake = Faker()
        Faker.seed(hash(row['email']))
        first_name = fake.first_name()
        last_name = fake.last_name()
        row['name'] = f"{first_name} {last_name}"
        row['email'] = fake_email(fake, first_name, last_name)
        row['phone'] = fake.phone_number()
        return row
    
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    email = Column(String)
    first_name = Column(String)
    last_name = Column(String)
    bio = Column(String)
    encrypted_password = Column(String)
    salt = Column(String)
    url = Column(String)
    confirmation_token = Column(String)
    remember_token = Column(String)
    
    def anonymize_row(row):
        fake = Faker()
        Faker.seed(row['id'])
        first_name = fake.first_name()
        last_name = fake.last_name()
        
        row['email'] = fake_email(fake, first_name, last_name)
        row['encrypted_password'] = fake.sha256()
        row['salt'] = fake.sha1()
        row['first_name'] = first_name
        row['last_name'] = last_name
        row['bio'] = fake.paragraph() if row['bio'] else None
        row['url'] = fake.url()
        row['confirmation_token'] = fake.sha1() if row['confirmation_token'] else None
        row['remember_token'] = fake.sha1() if row['remember_token'] else None
        return row

def chunk_dataframe(df, chunkSize = 10000): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf
    
def anonymize_table(table_class):
    name = f"{table_class.__name__} Table"
    display(HTML(f"<h2>{name}</h2>"))
    session = Session()
    df = pd.read_sql(session.query(table_class).statement, session.bind)
    display(HTML(f"<h4>Anonymizing...</h4>"))
    anonymized = df.parallel_apply(table_class.anonymize_row, axis=1) if len(df) > 10000\
      else df.progress_apply(table_class.anonymize_row, axis=1)
    display(HTML(f"<h4>Updating database table...</h4>"))
    chunks = chunk_dataframe(anonymized, 2500
                            )
    pb = tqdm(total=len(anonymized))
    for chunk in chunks:
        session.bulk_update_mappings(table_class, chunk.to_dict('records'))
        session.flush()
        session.commit()
        pb.update(len(chunk))
    pb.close()
    return anonymized

def save_snapshot():
    display(HTML(f"<h2>Saving Database Snapshot</h2>"))
    outdir = f"{os.getcwd()}/snapshots"
    tmpfile = NamedTemporaryFile(dir=outdir, suffix=".pgdump")
    display(HTML(f"<h4>Writing to {tmpfile.name}</h4>"))
    !pg_dump -U {db_user} -h {db_host} -d {db_name} -Fc --file {tmpfile.name}

    with open(tmpfile.name, 'rb') as infile:
        hash = hashlib.sha256(infile.read()).hexdigest()
        renamed_path = f"{os.getcwd()}/snapshots/{hash}.anonymized.pgdump"
        display(HTML(f"<h4>Copying to {renamed_path}</h4>"))
        shutil.copy(tmpfile.name, renamed_path)
        
    display(HTML(f"<h4>Done</h4>"))

anonymized_invitations = anonymize_table(Invitation)
anonymized_users = anonymize_table(User)
anonymized_projects = anonymize_table(Project)

save_snapshot()